In [ ]:
import os
from openai import AzureOpenAI
import argparse
import sys
import json
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

#Load environment variables
from dotenv import load_dotenv
load_dotenv()

ImportError: cannot import name 'PrioritizedFields' from 'azure.search.documents.indexes.models' (c:\sources\gh\firewall-agent-checker\.venv\Lib\site-packages\azure\search\documents\indexes\models\__init__.py)

In [15]:
#instatiate Azure OpenAI client
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

model = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")


In [16]:
#Test connection
response = client.chat.completions.create(
    model=model, # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

Yes, several Azure AI services support **Customer Managed Keys (CMK)**, allowing customers to have greater control over the encryption keys used to secure their data. Here are some examples of Azure AI services that support CMK:

### 1. **Azure Cognitive Services**
   Many Azure Cognitive Services—including services like Text Analytics, Translator, Computer Vision, and others—support customer managed keys for encrypting data at rest. This gives customers the ability to bring their own encryption keys stored in Azure Key Vault.

### 2. **Azure Machine Learning**
   Azure Machine Learning supports customer managed keys to encrypt workspace and data assets, including datasets, datastores, machine learning models, notebooks, and more. By using CMK, organizations can manage the lifecycle of their encryption keys and enforce security policies.

### 3. **Azure Synapse Analytics (with AI integration)**
   Azure Synapse Analytics, which often integrates Azure AI capabilities (like cognitive ser

In [11]:
# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Update the file path to the correct location
file_path = r"C:\sources\gh\firewall-agent-checker\data\general_network_risks_ports_public.json"

# Read the file and load its content into a string
with open(file_path, 'r') as file:
    port_risks_public = file.read()

print(port_risks_public)

[
 {
   "risk_id": "R001",
   "cve_id": "",
   "description": "Active Users - On some UNIX machines, creating a TCP connection to this port will dump the active processes and who launched them.",
   "cvss_score": 5,
   "ports": 11,
   "protocols": "tcp",
   "ip_ranges": "0.0.0.0/0",
   "source": "Predefined",
   "last_updated": "03/03/2025"
 },
 {
   "risk_id": "R002",
   "cve_id": "",
   "description": "FTP Data- Unencrypted file transfer",
   "cvss_score": 5,
   "ports": 20,
   "protocols": "tcp",
   "ip_ranges": "0.0.0.0/0",
   "source": "Predefined",
   "last_updated": "03/03/2025"
 },
 {
   "risk_id": "R003",
   "cve_id": "",
   "description": "FTP Control- Unencrypted file transfer",
   "cvss_score": 5,
   "ports": 21,
   "protocols": "tcp",
   "ip_ranges": "0.0.0.0/0",
   "source": "Predefined",
   "last_updated": "03/03/2025"
 },
 {
   "risk_id": "R004",
   "cve_id": "",
   "description": "SSH - Brute force risk is exposed",
   "cvss_score": 7.5,
   "ports": 22,
   "protocols":

In [12]:
prompt = f"""
Perform a comprehensive risk assessment of the existing firewall rules for an Azure Firewall. You will be provided with two CSV files:

Your task is:
- To evaluate all provided firewall rules against the known risks and risky ports.
- Tp identify any potential vulnerabilities or misconfigurations
- To Provide a complete report with all your findings about all the risky rules. do not truncate the report.

Instructions:
- For each line from the Firewall Rules for the assessment CSV:evaluate the risk of the firewall rule according to the risk contained in the Known Risks and Risky Ports and IPs CSV
- High Risk Rules have cvss_score greater or equal than 7, up to 10
- Medium Risk Rules have the cvss_score between 4 and 7
- Low Risk Rules have cvss_score than less than 4
- Ensure that your evaluation is thorough and based on the information provided in the CSV files.
- Avoid introducing unsupported information or speculation.
Output in the form of report:
- Do not escape any of the found risky rules, even if the list is long
- Provide content stricktly following this format:
    -The report header contains the Overall Risk Summary: total number of validated rules, overall risk level with number of risky rules split by the severity: High Risk, Medium Risk, Low Risk
    - The report body contains the Risk Summary Table containing ALL found risky rules with columns: Rule Name, Risk Level, Risk Details,Recommendations. Remember to put all rules that were considered as risky, do not skip any. Order by the risk High, Medium, Low.
    - Do not provide any other content, such as Recommendations Overview, Conclusion, etc.
- Provide a full un-trancated report in Markdown format.

Known Risks and Risky Ports and IPs:
{port_risks_public}
"""


In [17]:
# Prepare the chat prompt
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are an AI assistant that helps people find information.\n"
                + prompt,
            }
        ],
    }
]

In [19]:
# Generate the completion
completion = client.chat.completions.create(
    model=model,
    messages=chat_prompt,
    max_tokens=16384,
    temperature=0,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
)

In [21]:
# Extract the content from the completion
report_content = completion.choices[0].message.content

# Print the formatted report
print("### Azure Firewall Rules Risk Assessment Report ###\n")
print(report_content)

### Azure Firewall Rules Risk Assessment Report ###

```markdown
# Azure Firewall Rules Risk Assessment Report

## Overall Risk Summary
- Total Number of Validated Rules: 15
- Overall Risk Level:
  - High Risk: 6 rules
  - Medium Risk: 5 rules
  - Low Risk: 4 rules

---

## Risk Summary Table

| Rule Name       | Risk Level   | Risk Details                                                                 | Recommendations                                                                 |
|------------------|--------------|------------------------------------------------------------------------------|---------------------------------------------------------------------------------|
| Telnet           | High Risk    | Risk ID: R005 - Unecnrypted, deprecated (Port: 23, Protocol: TCP, CVSS: 9)  | Disable Telnet access and use encrypted protocols like SSH.                     |
| SMTP             | High Risk    | Risk ID: R006 - Often targeted for spam and phishing attacks (Port: 25, Protocol

In [29]:
# Configure environment variables for Azure Cognitive Search
search_service_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
search_index_name = os.getenv("AZURE_AI_SEARCH_INDEX")
search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")
credential = AzureKeyCredential("AZURE_AI_SEARCH_API_KEY")

In [30]:
# Initialize the Azure Cognitive Search client
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=search_index_name,
    credential=AzureKeyCredential(search_api_key)
)

In [31]:
# Create the index client
index_client = SearchIndexClient(endpoint=search_service_endpoint, credential=credential)

In [87]:
system_message = """Assistant supports users to determine whether a firewall rule is risky or not.
You have access to an Azure Cognitive Search index with a list of ports and IP ranges that are classified as risky. You can search for entries by "ip_ranges"
You are designed to be an interactive assistant, so you can ask users clarifying questions to help them determine whether the firewall rule is risky. It's better to give more detailed queries to the search index rather than vague one.
"""

messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": "Help me find out whether this port is risky or not: 20"}]

#"role": "user", "content": "Help me find out whether this firewall rule is risky or not: 193.163.125.240"

tools = [
    {
        "type": "function",
        "function": {
            "name": "query_risky_ip_ranges",
            "description": "Retrieve risky IP ranges from Azure AI Search index",
            "parameters": {
                "type": "object",
                "properties": {
                    "ip_range": {
                        "type": "string",
                        "description": "The ip-range from the firewall rule request to search for in the risky IP range index",
                    },
                },
                "required": ["ip_range"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "query_risky_ports",
            "description": "Retrieve risky ports from Azure AI Search index",
            "parameters": {
                "type": "object",
                "properties": {
                    "port": {
                        "type": "string",
                        "description": "The port number from the firewall rule request to search for in the risky ports index",
                    },
                },
                "required": ["port"],
            },
        },
    },
]

response = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
    temperature=0.2,
    tool_choice="auto",
)

print(response.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uNFOiCFYx8bptfFSVLI0ZPNm', function=Function(arguments='{"port":"20"}', name='query_risky_ports'), type='function')])


In [88]:
def query_risky_ip_ranges(ip_range):
    results = search_client.search(
        search_text=ip_range,  # Perform a keyword search using the provided IP range
        query_type="full",  # Use simple query type for keyword search
        search_fields=["ip_ranges"],  # Field to search for the IP range
        select=["risk_id", "description", "cvss_score", "ports", "protocols", "ip_ranges", "last_updated"],  # Select relevant fields
        top=1  # Limit to top 1 results
    )

    top_result = next(iter(results), None)
    if top_result:
        return json.dumps({
            "risk_id": top_result["risk_id"],
            "description": top_result["description"],
            "cvss_score": top_result["cvss_score"],
            "ports": top_result["ports"],
            "protocols": top_result["protocols"],
            "ip_ranges": top_result["ip_ranges"],
            "last_updated": top_result["last_updated"]
        }, indent=2)

    return "No risky IP range found for this input."

In [89]:
def query_risky_ports(port):
    results = search_client.search(
        search_text=str(port),  # Perform a keyword search using the provided port
        query_type="full",  # Use simple query type for keyword search
        search_fields=["ports"],  # Field to search for the port
        select=["risk_id", "description", "cvss_score", "ports", "protocols", "ip_ranges", "last_updated"],  # Select relevant fields
        top=1  # Limit to top 1 results
    )

    top_result = next(iter(results), None)
    if top_result:
        return json.dumps({
            "risk_id": top_result["risk_id"],
            "description": top_result["description"],
            "cvss_score": top_result["cvss_score"],
            "ports": top_result["ports"],
            "protocols": top_result["protocols"],
            "ip_ranges": top_result["ip_ranges"],
            "last_updated": top_result["last_updated"]
        }, indent=2)

    return "No risky port found for this input."

In [ ]:
test_ports_risky = query_risky_ports("20")
print(test_ports_risky)

'{\n  "risk_id": "R002",\n  "description": "FTP Data- Unencrypted file transfer",\n  "cvss_score": "5.0",\n  "ports": "20",\n  "protocols": "tcp",\n  "ip_ranges": "0.0.0.0/0",\n  "last_updated": "2025-03-03T00:00:00Z"\n}'

In [ ]:
test_ipranges_risky = query_risky_ip_ranges("193.163.125.241")
print(test_ipranges_risky)

'No risky IP range found for this input.'

In [ ]:
test_ipranges_unrisky = query_risky_ip_ranges("193.163.125.245")
print(test_ipranges_unrisky)

In [ ]:
test_ports_unrisky = query_risky_ports("145")
print(test_ports_unrisky)

In [91]:
def run_conversation(messages, tools, available_functions):

    # Step 1: send the conversation and available functions to GPT
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        temperature=0.2,
    )
    response_message = response.choices[0].message

    # Step 2: check if the model wants to call a function
    if response_message.tool_calls:
        print("Recommended Function call:")
        print(response_message.tool_calls[0])
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        function_name = response_message.tool_calls[0].function.name

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        try:
            function_args = json.loads(response_message.tool_calls[0].function.arguments)
        except json.JSONDecodeError as e:
            return f"Failed to parse function arguments: {e}"

        try:
            function_response = function_to_call(**function_args)
        except Exception as e:
            return f"Error while calling '{function_name}': {e}"

        #function_args = json.loads(response_message.tool_calls[0].function.arguments)
        #function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to the model

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message.role,
                "function_call": {
                    "name": response_message.tool_calls[0].function.name,
                    "arguments": response_message.tool_calls[0].function.arguments,
                },
                "content": None,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = client.chat.completions.create(
            messages=messages,
            model=model,
        )  # get a new response from GPT where it can see the function response

        return second_response
    else:
        return response

In [94]:
system_message = """Assistant is a large language model designed to help determine whether a firewall rule is risky or not.
You have access to an Azure Cognitive Search index with a list of ports and IP ranges that are classified as risky. You can search for entries by "ip_range" and "port". Perform consecutive queries, first checking for the IP range and then for the port"
You are designed to be an interactive assistant, so you can ask users clarifying questions to help them determine whether the firewall rule is risky. It's better to give more detailed queries to the search index rather than vague one.
If cvss_score >= 7.0, classify the IP/rule as High Risk
If cvss_score between 4.0 and 6.9, classify as Medium Risk
If cvss_score < 4.0, classify as Low Risk
"""

messages = [
    {"role": "system", "content": system_message},
    {
        "role": "user",
        "content": "Help me find out whether this firewall rule is risky or not: ip-range is 193.163.125.245, port is 20, protocol is TCP",
    },
]

available_functions = {
    "query_risky_ip_ranges": query_risky_ip_ranges,
    "query_risky_ports": query_risky_ports,
}

result = run_conversation(messages, tools, available_functions)

print("Final response:")
if isinstance(result, str):
    print(result)  # It's an error message or string content
else:
    print(result.choices[0].message.content)  # It's a ChatCompletionMessage object
#print(result.choices[0].message.content)

Recommended Function call:
ChatCompletionMessageToolCall(id='call_RpbXPAYvprlK7K1XQnKD4rID', function=Function(arguments='{"ip_range": "193.163.125.245"}', name='query_risky_ip_ranges'), type='function')

Output of function call:
No risky IP range found for this input.

Messages in second request:
{'role': 'system', 'content': 'Assistant is a large language model designed to help determine whether a firewall rule is risky or not.\nYou have access to an Azure Cognitive Search index with a list of ports and IP ranges that are classified as risky. You can search for entries by "ip_range" and "port". Perform consecutive queries, first checking for the IP range and then for the port"\nYou are designed to be an interactive assistant, so you can ask users clarifying questions to help them determine whether the firewall rule is risky. It\'s better to give more detailed queries to the search index rather than vague one.\nIf cvss_score >= 7.0, classify the IP/rule as High Risk\nIf cvss_score bet